In [1]:
using GynC
using JLD

In [2]:
include("../scripts/gyncplots.jl");

In [3]:
h=0.5
niter=5000
grad = true

function optimize(m, r) 
  w0=GynC.uniformweights(m.xs|>length);
  if grad 
    GynC.mple(m, w0, niter, r, h)
  else
    [GynC.optimineq(m,r,w0)] 
  end
end

optimize (generic function with 1 method)

In [4]:
function getsamples(thin=10000)
  ss = GynC.readsamples("/datanumerik/bzfsikor/gync/master");
  all = vcat([s.samples for s in ss]...);
  samples = all[1:thin:end,:];
  @show size(samples,1)
  samples
end

function getmodels(samples, zmult)
  @time m = GynC.gyncmodel([samples[i,:] for i=1:size(samples,1)], GynC.alldatas(), zmult = zmult)
  @time ms = GynC.smoothedmodel(m, zmult)
  n = length(m.xs)
  @time muni = GynC.gyncmodel(vcat(GynC.samplepi0(n*5, trajts), m.xs), m.datas, zmult=0);
  m, ms, muni
end


function getweights(m, ms, muni)
  ws = Dict()
  @time ws["uni"] = [inverseweights(muni.xs)]
  @time ws["MPLE"] = optimize(m, 100/101)
  @time ws["NPMLE"] = optimize(m, 0)
  @time ws["DS-MLE"] = optimize(ms, 0);
  ws
end

getweights (generic function with 1 method)

In [7]:
@load "paperplots/$(niter)iterh05.jld" m ms muni ws

4-element Array{Symbol,1}:
 :m   
 :ms  
 :muni
 :ws  

In [18]:
muni = GynC.gyncmodel(GynC.samplepi0(3000, trajts), m.datas, zmult=0)

GynC.LikelihoodModel(Array{Float64,1}[[7482.87,24316.6,625.325,10.1485,0.0200154,0.417625,0.000453428,3.88467,8.31965,76.4304  …  2.21184,0.729829,98.6764,103.031,0.0577993,0.00888676,0.00100052,0.000118478,-4.66955e-5,30.0],[27893.7,26623.4,703.709,10.8675,0.0103428,0.358266,0.0011153,1.35642,3.24448,287.203  …  -1.8324,0.447055,50.0495,71.8242,0.00996018,0.00879177,0.000958289,7.08255e-5,-0.000101283,30.0],[23585.1,34354.1,162.606,9.86929,0.0112635,0.92886,2.32681e-6,16.1791,9.9992,83.8603  …  -7.26203,-1.32654,69.2617,58.8559,0.00694054,0.00890776,0.00103168,0.000117253,2.82667e-5,30.0],[637.147,16521.7,371.195,6.41146,0.0105301,0.215293,0.000890641,9.70139,5.51239,79.9926  …  12.284,0.45953,15.9688,97.4725,0.0135715,0.00910563,0.000895764,0.000114941,1.12768e-5,30.0],[21830.1,6781.73,636.508,10.4938,0.0091097,0.525586,0.00137669,2.58231,6.35926,98.1485  …  5.24414,4.56806,150.789,61.2902,0.00758332,0.00895604,0.00100235,0.000100531,7.86024e-5,30.0],[24579.7,6578.27,158.029,6.32676,

In [19]:
@save "paperplots/munionly.jld" muni

In [ ]:
m,ms,muni = getmodels(getsamples(), 50)
ws = getweights(m,ms,muni)

In [8]:
@save "paperplots/grad$(grad)h$(h)niter$(niter).jld" m ms muni ws

In [7]:
ws

Dict{Any,Any} with 4 entries:
  "MPLE"   => Any[[0.000377644,0.000377644,0.000377644,0.000377644,0.000377644,…
  "NPMLE"  => Any[[0.000377644,0.000377644,0.000377644,0.000377644,0.000377644,…
  "uni"    => Array{Float64,1}[[6.72914e-5,6.73158e-5,6.71313e-5,6.72068e-5,6.7…
  "DS-MLE" => Any[[0.000377644,0.000377644,0.000377644,0.000377644,0.000377644,…